In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
dataset = pd.read_csv('data/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


### Independent and Dependent Variable

In [3]:
X = dataset.iloc[:,3:13]
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [4]:
y = dataset.iloc[:,13]
y.head()

0    1
1    0
2    1
3    0
4    0
Name: Exited, dtype: int64

### One Hot Encoding

In [6]:
X = pd.get_dummies(X,drop_first=True)
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_Germany,Geography_Spain,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


### Train Test Split

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [10]:
print(X_train.shape,X_test.shape)
print(y_train.shape,y_test.shape)

(8000, 11) (2000, 11)
(8000,) (2000,)


### Feature Selection

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

### Performing Hyperparameter Tuning

In [12]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [26]:
from keras.models import Sequential
from keras.layers import LeakyReLU,Dense,Activation,Embedding,Flatten,BatchNormalization,Dropout
from keras.activations import sigmoid,relu

In [40]:
def create_model(layers,activation):
    model = Sequential()
    for i,nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
    model.add(Dense(units=1,activation='sigmoid',kernel_initializer='glorot_uniform'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model
model = KerasClassifier(build_fn=create_model,verbose=0)
layers = [(20),(40,20),(45,30,15)]
activation = ['sigmoid','relu']
param_grid  = dict(layers=layers,activation=activation,batch_size=[128,256],epochs=[30])
grid = GridSearchCV(estimator=model,param_grid=param_grid,cv=5)
grid_result = grid.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
TypeError: 'int' object is not iterable

  FitFailedWarning)


### Best Results and Parameters

In [42]:
print(grid_result.best_score_)
print(grid_result.best_params_)

0.8547500014305115
{'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': (40, 20)}


### Prediction

In [45]:
y_pred = (grid.predict(X_test) > 0.5)

In [48]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [49]:
matrix = confusion_matrix(y_pred,y_test)
score  = accuracy_score(y_pred,y_test)

In [50]:
matrix

array([[1554,  230],
       [  41,  175]], dtype=int64)

In [51]:
score

0.8645